In [8]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from sklearn.model_selection import KFold

In [9]:
# 폴더에서 X, y 데이터 불러옴

images = []
labels = []

folder_paths = ["./able_with_rotation", "./unable_with_rotation", "./able_with_rotation_toneup", "./unable_with_rotation_toneup"]
label_value = [1, 0, 1, 0]

for folder_path, label in zip(folder_paths, label_value):
    for filename in os.listdir(folder_path):
        full_path = os.path.join(folder_path, filename)
        img = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
        if img is not None and img.shape == (80, 80):
            img = cv2.resize(img, dsize=(50, 50))
            images.append(img)
            labels.append(label)


images = np.array(images)
labels = np.array(labels)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [11]:
# CNN 모델 정의
location_model = models.Sequential([
    # Convolutional Layer 1
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(50, 50, 1)),
    layers.MaxPooling2D((2, 2)),

    # Convolutional Layer 2
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    # Flattening Layer
    layers.Flatten(),

    # Fully Connected Layer 1
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),

    # Fully Connected Layer 2
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),

    # Output Layer
    layers.Dense(1, activation='sigmoid')  # 이진 분류를 위한 sigmoid 활성화 함수
])

# 모델 요약 출력
location_model.summary()


c:\Users\gunwo\miniconda3\envs\mlenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 50, 50, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 25, 25, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 25, 25, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       589,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 610,817 (2.33 MB)

 Trainable params: 610,817 (2.33 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
location_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
k = 5
kf = KFold(n_splits=k, shuffle=True)

fold_no = 1
accuracies = []

for train_index, val_index in kf.split(X_train):
    print(f'Training fold {fold_no}')

    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    history = location_model.fit(X_train_fold, y_train_fold, epochs=15, validation_data=(X_val_fold, y_val_fold))

    scores = location_model.evaluate(X_val_fold, y_val_fold, verbose=0)
    print(f'Fold {fold_no} accuracy: {scores[1]}')
    accuracies.append(scores[1])

    fold_no += 1

print(f'Average accuracy across all folds: {np.mean(accuracies)}')

Training fold 1
Epoch 1/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.6103 - loss: 3.7317 - val_accuracy: 0.8573 - val_loss: 0.3436
Epoch 2/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.8288 - loss: 0.3746 - val_accuracy: 0.9028 - val_loss: 0.2449
Epoch 3/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.8702 - loss: 0.3109 - val_accuracy: 0.9151 - val_loss: 0.2227
Epoch 4/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.8730 - loss: 0.2852 - val_accuracy: 0.9164 - val_loss: 0.2448
Epoch 5/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.9155 - loss: 0.2216 - val_accuracy: 0.9262 - val_loss: 0.2058
Epoch 6/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.9382 - loss: 0.1718 - val_accuracy: 0.9373 - val_loss: 0.1984
Epoch 7/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.9384 - loss: 0.1669 - val_accuracy: 0.9508 - val_loss: 0.1468
Epoch 8/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.9511 - loss: 0

In [14]:
def evaluate_model(location_model, X_test, y_test):
    test_loss, test_accuracy = location_model.evaluate(X_test, y_test, verbose=2)
    print(f"Test loss: {test_loss:.4f}")
    print(f"Test accuracy: {test_accuracy:.4f}")

    return test_loss, test_accuracy

In [15]:
test_loss, test_accuracy = evaluate_model(location_model, X_test, y_test)
print(f'test loss : {test_loss}')
print(f'test accuracy : {test_accuracy}')
print(X_test.shape)

32/32 - 0s - 12ms/step - accuracy: 0.9783 - loss: 0.2334
Test loss: 0.2334
Test accuracy: 0.9783
test loss : 0.2334030270576477
test accuracy : 0.9783464670181274
(1016, 50, 50)


In [16]:
location_model.save('CNN_location_model.keras')
print('model saved!')

model saved!
